## Set up the enviroment - Flex and Bison
- Install Python
- Setup Jupyter Lab/Notes)
- If not on a Unix-ish platform (Unix, Linux, BSD, etc), install a unix shell for bash commands to work
- Install gcc (gnu c compiler) or any other C compiler that works in your platform
- Install Flex (e.g. pip install flex)
- Install Bison (e.g. pip install bison)

## (1) Word counting example

In [1]:
%%file ch1-1.l

/* ch1-1.l just like unix wc */

%{
int chars = 0;
int words = 0;
int lines = 0;
%}

%%

[^ \t\n\r\f\v]+ { words++; chars += strlen(yytext); }
\n		{ chars++; lines++; }
.		{ chars++; }

%%

int main(void)
{
    yylex();
    printf("Lines:%3d\nWords:%3d\nChars:%3d\n", lines, words, chars);
    return 0;
}


Writing ch1-1.l


In [2]:
%%bash
flex ch1-1.l
gcc lex.yy.c -ll

ld: warning: object file (/Library/Developer/CommandLineTools/SDKs/MacOSX14.2.sdk/usr/lib/libl.a[arm64][3](libyywrap.o)) was built for newer 'macOS' version (14.2) than being linked (14.0)


In [3]:
%%bash
./a.out
non sine periculo
tempus fugit carpe diem

Lines:  2
Words:  7
Chars: 42


## (2) Mapping some words from British to American english using Flex

In [4]:
%%file ch1-2.l

%%
"colour" { printf("color"); }
"flavour" { printf("flavor"); }
"clever" { printf("smart"); }
"smart" { printf("elegant"); }
"liberal" { printf("conservative"); }
. { printf("%s", yytext); }
%%

int main(void)
{
    yylex();
    return 0;
}

yywrap() { return 1; }


Writing ch1-2.l


In [5]:
%%bash
flex ch1-2.l
gcc lex.yy.c -ll 

ch1-2.l:17:1: warning: type specifier missing, defaults to 'int'; ISO C99 and later do not support implicit int [-Wimplicit-int]
yywrap() { return 1; }
^
int
1 warning generated.


In [6]:
%%bash
./a.out
liberal
clever
smart

conservative
smart
elegant


## (3) Recognise tokens for the calculator and print them out - scanner -  Flex

In [7]:
%%file ch1-3.l

%%
"+"	{ printf("PLUS\n"); }
"-"	{ printf("MINUS\n"); }
"*"	{ printf("TIMES\n"); }
"/"	{ printf("DIVIDE\n"); }
"|"     { printf("ABS\n"); }
[0-9]+	{ printf("NUMBER %s\n", yytext); }
\n      { printf("NEWLINE\n"); }
[ \t] { }
.	{ printf("Mystery character %s\n", yytext); }
%%


Writing ch1-3.l


In [8]:
%%bash
flex ch1-3.l
gcc lex.yy.c -ll 

ld: warning: object file (/Library/Developer/CommandLineTools/SDKs/MacOSX14.2.sdk/usr/lib/libl.a[arm64][2](libmain.o)) was built for newer 'macOS' version (14.2) than being linked (14.0)
ld: warning: object file (/Library/Developer/CommandLineTools/SDKs/MacOSX14.2.sdk/usr/lib/libl.a[arm64][3](libyywrap.o)) was built for newer 'macOS' version (14.2) than being linked (14.0)


In [9]:
%%bash
./a.out
12+34
540 - 267

NUMBER 12
PLUS
NUMBER 34
NEWLINE
NUMBER 540
MINUS
NUMBER 267
NEWLINE


## (4) Recognize tokens for the calculator and print them out
### Extending the scanner to map to a token identifier

In [10]:
%%file ch1-4.l


%{
   enum yytokentype {
     NUMBER = 258,
     ADD = 259,
     SUB = 260,
     MUL = 261,
     DIV = 262,
     ABS = 263,
     EOL = 264 /* end of line */
   };

   int yylval;

%}

%%

"+"	{ return ADD; }
"-"	{ return SUB; }
"*"	{ return MUL; }
"/"	{ return DIV; }
"|"     { return ABS; }
[0-9]+	{ yylval = atoi(yytext); return NUMBER; }
\n      { return EOL; }
[ \t]   { /* ignore white space */ }
.	{ printf("Mystery character %c\n", *yytext); }
%%

int main()
{
  int tok;

  while(tok = yylex())
  {
    printf("%d", tok);
    
    if(tok == NUMBER) 
        printf(" = %d\n", yylval);
    else 
        printf("\n");
  }
  return 0;
}


Writing ch1-4.l


In [11]:
%%bash
flex ch1-4.l
gcc lex.yy.c -ll 

ch1-4.l:35:13: warning: using the result of an assignment as a condition without parentheses [-Wparentheses]
  while(tok = yylex())
        ~~~~^~~~~~~~~
ch1-4.l:35:13: note: place parentheses around the assignment to silence this warning
  while(tok = yylex())
            ^
        (            )
ch1-4.l:35:13: note: use '==' to turn this assignment into an equality comparison
  while(tok = yylex())
            ^
            ==
1 warning generated.
ld: warning: object file (/Library/Developer/CommandLineTools/SDKs/MacOSX14.2.sdk/usr/lib/libl.a[arm64][3](libyywrap.o)) was built for newer 'macOS' version (14.2) than being linked (14.0)


In [12]:
%%bash
./a.out
a / 34 + |45

Mystery character a
262
258 = 34
259
263
258 = 45
264


## (5) Calculator example - Using Flex to scan the tokens and Bison for rules

In [37]:
%%file ch1-5.l

/* recognize tokens for the calculator and print them out */

%{
# include "ch1-5.tab.h"
%}

%%
"+"	{ return ADD; }
"-"	{ return SUB; }
"*"	{ return MUL; }
"/"	{ return DIV; }
"|"     { return ABS; }
"("     { return OP; }
")"     { return CP; }
[0-9]+	{ yylval = atoi(yytext); return NUMBER; }
\n      { return EOL; }
"//".*  
[ \t]   { /* ignore white space */ }
%%


Overwriting ch1-5.l


In [27]:
%%file ch1-5.y

/* Implement parser using Bison rules/gammar */

%{
   #include <stdio.h>
   void yyerror(const char* msg) {
      fprintf(stderr, "%s\n", msg);
   }
   int yylex();
%}


/* declare tokens */
%token NUMBER
%token ADD SUB MUL DIV ABS
%token OP CP
%token EOL

%%

calclist: /* nothing */
 | calclist exp EOL { printf("= %d\n> ", $2); }
 | calclist EOL { printf("> "); } /* blank line or a comment */
 ;

exp: factor
 | exp ADD exp { $$ = $1 + $3; }
 | exp SUB factor { $$ = $1 - $3; }
 | exp ABS factor { $$ = $1 | $3; }
 ;

factor: term
 | factor MUL term { $$ = $1 * $3; }
 | factor DIV term { $$ = $1 / $3; }
 ;

term: NUMBER
 | ABS term { $$ = $2 >= 0? $2 : - $2; }
 | OP exp CP { $$ = $2; }
 ;
%%

int main()
{
  printf("> "); 
  yyparse();
  return 0;
}



Overwriting ch1-5.y


In [ ]:
%%bash

bison -d ch1-5.y
flex ch1-5.l
gcc -o ch1-5 ch1-5.tab.c lex.yy.c -ll


In [43]:
%%bash
./ch1-5
2 + 3 * 5
100 - 50 + 25
50 / 5 * 2

> = 17
> = 75
> = 20
> 